In [14]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from tqdm import tqdm
import numpy as np
import math

-> SPOTIPY_CLIENT_ID - Get this from the Spotify app in the Developer Console

-> SPOTIPY_CLIENT_SECRET - Get this from the Spotify app in the Developer Console

-> SPOTIPY_REDIRECT_URI - Set to http://localhost:3000/callback

In [2]:
# To get client ID and client secret
client_id = '9ff8454bd3e049b6a1229206ce536ac8'  # <-- replace with your own client ID
client_secret = '24c245fb45fa46eb8c57d36dd8770b3d' # <-- replace with your own client secret

auth_manager = SpotifyOAuth(client_id = client_id, redirect_uri='http://localhost:3000/callback',
                            client_secret = client_secret, scope="playlist-modify-public", open_browser=False)
auth_url = auth_manager.get_authorize_url()

In [3]:
print(f"Please visit this URL to authorize the application: {auth_url}")
response = input("Paste the above link into your browser's address bar: ")

# Get access token using the obtained authorization code
token_info = auth_manager.get_access_token(response, as_dict = False)
print(token_info)

Please visit this URL to authorize the application: https://accounts.spotify.com/authorize?client_id=9ff8454bd3e049b6a1229206ce536ac8&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A3000%2Fcallback&scope=playlist-modify-public
BQAvKRWS8UqA-eaDefqVXew4YMzJPp_jBZu_Pox5k3cxnUBv_MKWr4Mu9qpm9o_ViRH0lW7gXbjW2o9xzHGT-FOyH7EPmtvvHnqgBUrQjEFjAyb6FlzM341c0XsRGBFqiF33aiFwKvCaC1nCRsxFToIGr3pohixHOKMb2gDNja8p1DvSPNxfEo4GZ4KVY5Th9MqUaTcbED2kwjgm7e-YkJpm2lvH4S26M2tFD-oaJFS9zq0BD9-A2sBXWlVqAdiJOn3fwdX160JyYdwA


In [4]:
spotify = spotipy.Spotify(auth_manager=auth_manager)
user_info = spotify.current_user()
print(user_info)

{'display_name': 'Jackson Chen', 'external_urls': {'spotify': 'https://open.spotify.com/user/31flqmapu6wd5a57mqga7pbgfobq'}, 'href': 'https://api.spotify.com/v1/users/31flqmapu6wd5a57mqga7pbgfobq', 'id': '31flqmapu6wd5a57mqga7pbgfobq', 'images': [], 'type': 'user', 'uri': 'spotify:user:31flqmapu6wd5a57mqga7pbgfobq', 'followers': {'href': None, 'total': 0}, 'country': 'US', 'product': 'free', 'explicit_content': {'filter_enabled': False, 'filter_locked': False}, 'email': 'jiaxinchen0519@gmail.com'}


In [13]:
# Read the song data, and apply some of them to make playlist according to years
df = pd.read_csv("song.csv")
years = df['year'].unique()
print(years)

[1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971
 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985
 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999
 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021 2022]


We see that the song data years spread from 1958 to 2022

In [7]:
pd.DataFrame(columns=['id','name','link']).to_csv("playlists.csv", index=False)

def is_playlist_created(playlist_name, existing_playlists):
    """
    Checks if playlist has already been created.

    Parameters:
    - playlist_name: str, name of the playlist to check
    - playlists_df: pd.DataFrame, DataFrame containing information about existing playlists

    Returns:
    - bool, True if the playlist has not been created, False otherwise
    """
    return not any(existing_playlists['name'] == playlist_name)

def save_playlists(playlist_info, playlist_name, existing_playlists):
    """
    Saves playlist information in a CSV file.

    Parameters:
    - playlist: dict, information about the newly created playlist
    - playlist_name: str, name of the playlist
    - playlists_df: pd.DataFrame, DataFrame containing information about existing playlists
    """
    new_playlist_entry = {
        'name': playlist_name,
        'id': playlist_info['id'],
        'link': playlist_info['external_urls']['spotify']
    }

    updated_playlists = pd.concat([existing_playlists, pd.DataFrame([new_playlist_entry])])
    updated_playlists.to_csv("playlists.csv", index=False)

def create_playlist(start_year, end_year):
    """
    Creates the playlist and adds tracks from songs.csv given the start and end year.

    Parameters:
    - start_year: int, starting year for creating the playlist
    - end_year: int, ending year for creating the playlist
    """
    # Read existing playlists information from playlists.csv
    existing_playlists = pd.read_csv("playlists.csv")

    # Generate playlist name based on the start and end years
    if (start_year - end_year) == 0:
        new_playlist_name = f"Top Songs: {start_year}"
    else:
        new_playlist_name = f"Top Songs: {start_year}-{end_year}"

    # Check if the playlist already exists
    if is_playlist_created(new_playlist_name, existing_playlists):
        # Create a new playlist 
        playlist = spotify.user_playlist_create(user=user_info['id'], name=new_playlist_name)

        # Get the Spotify URIs of tracks within the specified year range
        track_uris = list(df[(df['year'] >= start_year) & (df['year'] <= end_year)]['spotify_uri'].values)

        # Split URIs into chunks of 100 (Spotify API limit)
        chunks = np.array_split(track_uris, math.ceil(len(track_uris) / 100))

        # Add tracks to the created playlist in chunks
        for uri in chunks:
            spotify.user_playlist_add_tracks(user=user_info['id'], playlist_id=playlist['id'], tracks=uri)

        # Save information about the created playlist
        save_playlists(playlist, new_playlist_name, existing_playlists)

In [10]:
# make playlist from 2012 to 2023
start_year = 2012 # <-- adjust the year range if you want
end_year = 2023 # <-- adkust the year range if you want
years = list(range(start_year, end_year))

year_ranges = []
for i in range(0,21):
    for start in years[:-i]:
        end = start + i
        year_ranges.append((start, end))

len(year_ranges)

55

In [11]:
# This produces around 1090 playlists if the range increases to 1958-2023
for y in tqdm(year_ranges):
    create_playlist(start_year = y[0], end_year = y[1])

  0%|          | 0/55 [00:00<?, ?it/s]

100%|██████████| 55/55 [00:33<00:00,  1.64it/s]
